# Read WLAS Dataset and analyze it

In [1]:
import pandas as pd
import numpy as np
import json
import os
from cv2 import cv2
import mediapipe as mp
import shutil
import warnings
import datetime
warnings.filterwarnings('ignore')

## 1. Load WLAS Dataset

In [2]:
wlas_df = pd.read_json('archive/WLASL_v0.3.json')

In [3]:
wlas_df.head()

,gloss,instances
0,book,"[{'bbox': [385, 37, 885, 720], 'fps': 25, 'fra..."
1,drink,"[{'bbox': [551, 68, 1350, 1080], 'fps': 25, 'f..."
2,computer,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
3,before,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
4,chair,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."


In [4]:
wlas_df.shape

(2000, 2)

## 2. Extract the available videos list from the dataset

In [5]:
def get_videos_ids(json_list):
    """
    function to check if the video id is available in the dataset
    and return the viedos ids of the current instance
    
    input: instance json list
    output: list of videos_ids
    
    """
    videos_list = []    
    for ins in json_list:
        video_id = ins['video_id']
        if os.path.exists(f'archive/videos/{video_id}.mp4'):
            videos_list.append(video_id)
    return videos_list

In [6]:
with open('archive/WLASL_v0.3.json', 'r') as data_file:
    json_data = data_file.read()

instance_json = json.loads(json_data)
# instances_df = pd.json_normalize()

In [7]:
# instance_json[0]['instances']

In [8]:
get_videos_ids(instance_json[0]['instances'])[0]

'69241'

In [9]:
len(get_videos_ids(instance_json[0]['instances']))

6

In [10]:
wlas_df['videos_ids'] = wlas_df['instances'].apply(get_videos_ids)

In [11]:
wlas_df.head()

,gloss,instances,videos_ids
0,book,"[{'bbox': [385, 37, 885, 720], 'fps': 25, 'fra...","[69241, 07069, 07068, 07070, 07099, 07074]"
1,drink,"[{'bbox': [551, 68, 1350, 1080], 'fps': 25, 'f...","[69302, 65539, 17710, 17733, 65540, 17734, 177..."
2,computer,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...","[12328, 12312, 12311, 12338, 12313, 12314, 123..."
3,before,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...","[05728, 05749, 05750, 05729, 05730, 65167, 057..."
4,chair,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...","[09848, 09869, 09849, 09850, 09851, 65328, 09854]"


In [12]:
wlas_df.videos_ids[0][0]

'69241'

## 3. Dataset Classes analysis

In [13]:
wlas_df['samples_num'] = wlas_df['videos_ids'].apply(len)

In [14]:
wlas_df.head()

,gloss,instances,videos_ids,samples_num
0,book,"[{'bbox': [385, 37, 885, 720], 'fps': 25, 'fra...","[69241, 07069, 07068, 07070, 07099, 07074]",6
1,drink,"[{'bbox': [551, 68, 1350, 1080], 'fps': 25, 'f...","[69302, 65539, 17710, 17733, 65540, 17734, 177...",15
2,computer,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...","[12328, 12312, 12311, 12338, 12313, 12314, 123...",14
3,before,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...","[05728, 05749, 05750, 05729, 05730, 65167, 057...",16
4,chair,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_...","[09848, 09869, 09849, 09850, 09851, 65328, 09854]",7


In [15]:
print("minimum number of samples for a word:", wlas_df['samples_num'].min())
print("maximum number of samples for a word:", wlas_df['samples_num'].max())

minimum number of samples for a word: 2
maximum number of samples for a word: 16


In [16]:
wlas_df[['gloss', 'samples_num']].groupby('samples_num').agg({"gloss":'count'})

,gloss
samples_num,
2,14
3,76
4,335
5,511
6,402
7,307
8,151
9,100
10,48


In [17]:
words_count = dict(wlas_df[['gloss', 'samples_num']].groupby('samples_num').apply(lambda x: ', '.join(x['gloss'])))

In [18]:
words_count[2]

'gloves, careless, wash face, curtain, grey, lamp, look at, meaning, post, propaganda, ski, smoking, stepfather, tv'

## 4. Prepare the dataframe for information extraction

In [19]:
wlas_df.drop(['instances', "samples_num"], inplace=True, axis=1)

In [20]:
wlas_df.shape

(2000, 2)

In [138]:
dataset_frames = pd.DataFrame(columns=['gloss', 'video_id', 'frames_num'])

In [139]:
for i in wlas_df.index:
    gloss = wlas_df['gloss'].iloc[i]
    videos_list = wlas_df['videos_ids'].iloc[i]
    for j, video in enumerate(videos_list):
        dataset_frames = dataset_frames.append({'gloss': gloss, 'video_id':video, 'frames_num':j}, ignore_index=True)

In [140]:
dataset_frames

,gloss,video_id,frames_num
0,book,69241,0
1,book,07069,1
2,book,07068,2
3,book,07070,3
4,book,07099,4
...,...,...,...
11975,wheelchair,63047,2
11976,wheelchair,63050,3
11977,whistle,63186,0
11978,whistle,63188,1


In [142]:
dataset_frames['frames_num'] = dataset_frames.apply(lambda row: len(os.listdir(os.path.join("archive", "keypoints", row['gloss'], str(row['frames_num'])))),axis=1)

In [144]:
dataset_frames.to_csv("gloss_videos_frames.csv")

## 5. Videos Processing and information extraction

### 5.1 create and test keypoints extraction function

In [22]:
mp_holistic_model = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils 

In [23]:
def detect_keypoints(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [24]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic_model.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(10, 194, 80), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(214, 200, 80), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic_model.POSE_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(90, 194, 80), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(230, 200, 80), thickness=2, circle_radius=4))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic_model.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(20, 194, 80), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(190, 200, 80), thickness=2, circle_radius=4))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic_model.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(20, 194, 80), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(190, 200, 80), thickness=2, circle_radius=4))

In [25]:
def extract_keypoints(results):
    face_landmarks = np.array([[lm.x, lm.y, lm.z] for lm in results.face_landmarks.landmark]).flatten()if results.face_landmarks else np.zeros(468*3)
    pose_landmarks = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    righ_hand_landmarks = np.array([[lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    left_hand_landmarks = np.array([[lm.x, lm.y, lm.z] for lm in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose_landmarks, face_landmarks, left_hand_landmarks, righ_hand_landmarks])

In [26]:
# cap = cv2.VideoCapture("archive/videos/00335.mp4")
# no_frames = 0
# success = success, frame = cap.read()
# with mp_holistic_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened() and success:
#         frame = cv2.resize(frame, (600, 600))
#         image, results = detect_keypoints(frame, holistic)
#         draw_landmarks(image, results)
#         cv2.imshow("cam", image)

#         success, frame = cap.read()
#         no_frames += 1
        
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
        
# cap.release()
# cv2.destroyAllWindows()

In [27]:
# no_frames

In [28]:
# print("face landmarks length:", len(results.face_landmarks.landmark))
# print("pose landmakrs length:", len(results.pose_landmarks.landmark))
# print("right hand landmakrs length:", len(results.right_hand_landmarks.landmark))
# print("left hand landmarks length:", len(results.left_hand_landmarks.landmark))

In [29]:
# face_landmarks.shape

In [30]:
# pose_landmarks.shape

In [31]:
# righ_hand_landmarks.shape

In [32]:
# left_hand_landmarks.shape

In [33]:
# keypoints = extract_keypoints(results)

In [34]:
# keypints.shape

### 5.1.2 cut the videos to small number of frames

In [ ]:
dataset_frames = pd.read_csv("gloss_videos_frames.csv")

In [152]:
def cut_videos(main_path, word, video_id, row_index, num_of_frames):
    print(f"processing video frames number >> Word:({word}), Video Id:({video_id}), data index -> ({row_index}).")
    output_path = os.path.join(main_path, f"videos_with_{num_of_frames}_frames")
    os.makedirs(output_path, exist_ok=True)
    
    cap = cv2.VideoCapture(os.path.join(main_path, str(video_id)+".mp4"))
    success = success, frame = cap.read()
    frame_number = 1
    frames_list = []
    while cap.isOpened() and success and frame_number <= num_of_frames:
        frame = cv2.resize(frame, (600, 600))
        frames_list.append(frame)
        success, frame = cap.read()

        frame_number += 1

#                 cv2.waitKey(1)
    if frame_number < num_of_frames:
        blank_image = np.zeros((600, 600, 3), np.uint8)
        blank_list = [blank_image] * (num_of_frames - frame_number)
        frames_list.extend(blank_list)

    out = cv2.VideoWriter(os.path.join(output_path, f'{video_id}.avi'),cv2.VideoWriter_fourcc(*'DIVX'), 20.0, (600, 600))

    for frm in frames_list:
        out.write(frm)
    out.release()


    cap.release()
    cv2.destroyAllWindows()

In [153]:
dataset_frames.apply(lambda row: cut_videos("archive/videos", row['gloss'], row['video_id'], row.name, 70), axis=1)

processing video frames number >> Word:(book), Video Id:(69241), data index -> (0).
processing video frames number >> Word:(book), Video Id:(07069), data index -> (1).
processing video frames number >> Word:(book), Video Id:(07068), data index -> (2).
processing video frames number >> Word:(book), Video Id:(07070), data index -> (3).
processing video frames number >> Word:(book), Video Id:(07099), data index -> (4).
processing video frames number >> Word:(book), Video Id:(07074), data index -> (5).
processing video frames number >> Word:(drink), Video Id:(69302), data index -> (6).
processing video frames number >> Word:(drink), Video Id:(65539), data index -> (7).
processing video frames number >> Word:(drink), Video Id:(17710), data index -> (8).
processing video frames number >> Word:(drink), Video Id:(17733), data index -> (9).
processing video frames number >> Word:(drink), Video Id:(65540), data index -> (10).
processing video frames number >> Word:(drink), Video Id:(17734), data

processing video frames number >> Word:(candy), Video Id:(08919), data index -> (96).
processing video frames number >> Word:(candy), Video Id:(08920), data index -> (97).
processing video frames number >> Word:(candy), Video Id:(08921), data index -> (98).
processing video frames number >> Word:(candy), Video Id:(65298), data index -> (99).
processing video frames number >> Word:(candy), Video Id:(08924), data index -> (100).
processing video frames number >> Word:(candy), Video Id:(65299), data index -> (101).
processing video frames number >> Word:(candy), Video Id:(65300), data index -> (102).
processing video frames number >> Word:(candy), Video Id:(08915), data index -> (103).
processing video frames number >> Word:(candy), Video Id:(08925), data index -> (104).
processing video frames number >> Word:(cousin), Video Id:(65415), data index -> (105).
processing video frames number >> Word:(cousin), Video Id:(13647), data index -> (106).
processing video frames number >> Word:(cousi

KeyboardInterrupt: 

### 5.2 create dataset directories

In [35]:
def create_word_folders(word, num_of_videos):
    print(f"create {word} directory.")
    os.makedirs(f"archive/keypoints/{word}", exist_ok=True)
    for i in range(num_of_videos):
        print(f"\tcreate video number ({i}) directory.")
        os.makedirs(f"archive/keypoints/{word}/{i}", exist_ok=True)

In [82]:
def max_number_of_frames(word, num_of_videos):
    frames_num_list = []
    print(f"counting {word} directory.")
    for i in range(num_of_videos):
        l = len(os.listdir(f"archive/keypoints/{word}/{i}"))
        print(f"\counting video number ({i}) directory >>> {l}.")
        frames_num_list.append(l)
    return np.array(frames_num_list)

In [ ]:
wlas_df.apply(lambda row: create_word_folders(row['gloss'], len(row['videos_ids'])), axis=1)

create book directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
	create video number (5) directory.
create drink directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
	create video number (5) directory.
	create video number (6) directory.
	create video number (7) directory.
	create video number (8) directory.
	create video number (9) directory.
	create video number (10) directory.
	create video number (11) directory.
	create video number (12) directory.
	create video number (13) directory.
	create video number (14) directory.
create computer directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number 

	create video number (11) directory.
create color directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
	create video number (5) directory.
	create video number (6) directory.
	create video number (7) directory.
create corn directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
	create video number (5) directory.
	create video number (6) directory.
	create video number (7) directory.
	create video number (8) directory.
	create video number (9) directory.
	create video number (10) directory.
	create video number (11) directory.
create cow directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) dir

	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
	create video number (5) directory.
	create video number (6) directory.
	create video number (7) directory.
	create video number (8) directory.
	create video number (9) directory.
create join directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
	create video number (5) directory.
	create video number (6) directory.
	create video number (7) directory.
create light directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
	create video number (5) directory.
	create video number (6) directory.
	create video number (7) directory.
create live directory.
	create video number (0) directory.
	create video number (1) direc

create moon directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
	create video number (5) directory.
	create video number (6) directory.
create move directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
	create video number (5) directory.
	create video number (6) directory.
	create video number (7) directory.
	create video number (8) directory.
	create video number (9) directory.
create near directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
	create video number (5) directory.
	create video number (6) directory.
	create video number (7) directory.
	create video number (8) direct

	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
	create video number (5) directory.
	create video number (6) directory.
create awful directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
	create video number (5) directory.
	create video number (6) directory.
	create video number (7) directory.
create awkward directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
	create video number (5) directory.
	create video number (6) directory.
create bacon directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) d

...

	create video number (4) directory.
	create video number (5) directory.
create less directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
create linguistics directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
	create video number (5) directory.
	create video number (6) directory.
create lonely directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
	create video number (5) directory.
create long directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.

	create video number (3) directory.
	create video number (4) directory.
create tiptoe directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
create title directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
create tooth directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
	create video number (5) directory.
create toothbrush directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
create total directory.
	create video number (0) directory.
	create video number (1) directory.
	create vi

	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
create technology directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
create temple directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
	create video number (5) directory.
create tender directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) directory.
	create video number (5) directory.
create thailand directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
	create video number (4) direc

	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
create cuba directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
create curtain directory.
	create video number (0) directory.
	create video number (1) directory.
create customer directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
create d directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
create damage directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) directory.
create dancer directory.
	create video number (0) directory.
	create video number (1) directory.
	create video number (2) directory.
	create video number (3) d

0       None
1       None
2       None
3       None
4       None
        ... 
1995    None
1996    None
1997    None
1998    None
1999    None
Length: 2000, dtype: object

In [93]:
x = [wlas_df.apply(lambda row: max_number_of_frames(row['gloss'], len(row['videos_ids'])), axis=1).values]

counting book directory.
\counting video number (0) directory >>> 75.
\counting video number (1) directory >>> 30.
\counting video number (2) directory >>> 68.
\counting video number (3) directory >>> 86.
\counting video number (4) directory >>> 87.
\counting video number (5) directory >>> 38.
counting drink directory.
\counting video number (0) directory >>> 77.
\counting video number (1) directory >>> 44.
\counting video number (2) directory >>> 70.
\counting video number (3) directory >>> 93.
\counting video number (4) directory >>> 46.
\counting video number (5) directory >>> 89.
\counting video number (6) directory >>> 81.
\counting video number (7) directory >>> 43.
\counting video number (8) directory >>> 91.
\counting video number (9) directory >>> 105.
\counting video number (10) directory >>> 40.
\counting video number (11) directory >>> 48.
\counting video number (12) directory >>> 48.
\counting video number (13) directory >>> 59.
\counting video number (14) directory >>> 29

\counting video number (0) directory >>> 79.
\counting video number (1) directory >>> 44.
\counting video number (2) directory >>> 45.
\counting video number (3) directory >>> 85.
\counting video number (4) directory >>> 90.
\counting video number (5) directory >>> 38.
\counting video number (6) directory >>> 35.
counting share directory.
\counting video number (0) directory >>> 52.
\counting video number (1) directory >>> 56.
\counting video number (2) directory >>> 46.
\counting video number (3) directory >>> 48.
\counting video number (4) directory >>> 89.
\counting video number (5) directory >>> 30.
\counting video number (6) directory >>> 128.
counting soon directory.
\counting video number (0) directory >>> 44.
\counting video number (1) directory >>> 86.
\counting video number (2) directory >>> 44.
\counting video number (3) directory >>> 32.
\counting video number (4) directory >>> 65.
\counting video number (5) directory >>> 62.
\counting video number (6) directory >>> 103.
\c

\counting video number (6) directory >>> 72.
\counting video number (7) directory >>> 38.
\counting video number (8) directory >>> 94.
counting deep directory.
\counting video number (0) directory >>> 86.
\counting video number (1) directory >>> 76.
\counting video number (2) directory >>> 113.
\counting video number (3) directory >>> 38.
\counting video number (4) directory >>> 38.
\counting video number (5) directory >>> 97.
\counting video number (6) directory >>> 51.
\counting video number (7) directory >>> 29.
counting deer directory.
\counting video number (0) directory >>> 91.
\counting video number (1) directory >>> 95.
\counting video number (2) directory >>> 195.
\counting video number (3) directory >>> 53.
\counting video number (4) directory >>> 79.
\counting video number (5) directory >>> 50.
\counting video number (6) directory >>> 45.
counting dentist directory.
\counting video number (0) directory >>> 32.
\counting video number (1) directory >>> 81.
\counting video numb

\counting video number (0) directory >>> 43.
\counting video number (1) directory >>> 69.
\counting video number (2) directory >>> 36.
\counting video number (3) directory >>> 36.
\counting video number (4) directory >>> 84.
\counting video number (5) directory >>> 39.
counting thirsty directory.
\counting video number (0) directory >>> 89.
\counting video number (1) directory >>> 83.
\counting video number (2) directory >>> 58.
\counting video number (3) directory >>> 124.
\counting video number (4) directory >>> 87.
\counting video number (5) directory >>> 77.
\counting video number (6) directory >>> 77.
\counting video number (7) directory >>> 68.
counting ticket directory.
\counting video number (0) directory >>> 91.
\counting video number (1) directory >>> 96.
\counting video number (2) directory >>> 87.
\counting video number (3) directory >>> 74.
\counting video number (4) directory >>> 43.
\counting video number (5) directory >>> 47.
counting top directory.
\counting video numb

\counting video number (1) directory >>> 41.
\counting video number (2) directory >>> 61.
\counting video number (3) directory >>> 57.
\counting video number (4) directory >>> 98.
counting expect directory.
\counting video number (0) directory >>> 128.
\counting video number (1) directory >>> 38.
\counting video number (2) directory >>> 50.
\counting video number (3) directory >>> 103.
\counting video number (4) directory >>> 50.
\counting video number (5) directory >>> 75.
counting fancy directory.
\counting video number (0) directory >>> 56.
\counting video number (1) directory >>> 64.
\counting video number (2) directory >>> 97.
\counting video number (3) directory >>> 95.
\counting video number (4) directory >>> 29.
\counting video number (5) directory >>> 38.
\counting video number (6) directory >>> 81.
counting favorite directory.
\counting video number (0) directory >>> 46.
\counting video number (1) directory >>> 67.
\counting video number (2) directory >>> 89.
\counting video 

\counting video number (0) directory >>> 51.
\counting video number (1) directory >>> 89.
\counting video number (2) directory >>> 80.
\counting video number (3) directory >>> 96.
\counting video number (4) directory >>> 57.
counting maximum directory.
\counting video number (0) directory >>> 102.
\counting video number (1) directory >>> 88.
\counting video number (2) directory >>> 33.
\counting video number (3) directory >>> 93.
\counting video number (4) directory >>> 35.
\counting video number (5) directory >>> 91.
counting me directory.
\counting video number (0) directory >>> 79.
\counting video number (1) directory >>> 59.
\counting video number (2) directory >>> 88.
\counting video number (3) directory >>> 27.
counting mexico directory.
\counting video number (0) directory >>> 83.
\counting video number (1) directory >>> 72.
\counting video number (2) directory >>> 43.
\counting video number (3) directory >>> 90.
\counting video number (4) directory >>> 86.
\counting video numbe

\counting video number (4) directory >>> 72.
counting p directory.
\counting video number (0) directory >>> 45.
\counting video number (1) directory >>> 126.
\counting video number (2) directory >>> 30.
\counting video number (3) directory >>> 80.
counting parachute directory.
\counting video number (0) directory >>> 90.
\counting video number (1) directory >>> 49.
\counting video number (2) directory >>> 70.
\counting video number (3) directory >>> 116.
counting paragraph directory.
\counting video number (0) directory >>> 94.
\counting video number (1) directory >>> 32.
\counting video number (2) directory >>> 91.
\counting video number (3) directory >>> 44.
\counting video number (4) directory >>> 92.
counting parallel directory.
\counting video number (0) directory >>> 58.
\counting video number (1) directory >>> 102.
\counting video number (2) directory >>> 46.
\counting video number (3) directory >>> 34.
\counting video number (4) directory >>> 81.
counting pay attention director

\counting video number (1) directory >>> 68.
\counting video number (2) directory >>> 28.
\counting video number (3) directory >>> 76.
counting mistake directory.
\counting video number (0) directory >>> 78.
\counting video number (1) directory >>> 86.
\counting video number (2) directory >>> 25.
\counting video number (3) directory >>> 80.
counting mock directory.
\counting video number (0) directory >>> 71.
\counting video number (1) directory >>> 92.
\counting video number (2) directory >>> 45.
counting moose directory.
\counting video number (0) directory >>> 46.
\counting video number (1) directory >>> 92.
\counting video number (2) directory >>> 31.
\counting video number (3) directory >>> 99.
counting mosquito directory.
\counting video number (0) directory >>> 88.
\counting video number (1) directory >>> 59.
\counting video number (2) directory >>> 95.
\counting video number (3) directory >>> 57.
counting motivate directory.
\counting video number (0) directory >>> 73.
\countin

In [113]:
out = np.concatenate(x[0]).ravel()
np.median(out)

70.0

In [112]:
count = np.bincount(out)
print(np.argmax(count))
print(out[85])

85
93


### 5.3 read the videos and write the keypoints to numpy file

In [ ]:
def create_np_keypoints_file(word, videos_list, row_index, holistic_model):
    print(f"extracting the keypoints for ({word}) of index -> ({row_index}).")
    start = datetime.datetime.now()

    for i in range(len(videos_list)):
        print(f"\textracting the keypoints from video number ({i}) ...")
        cap = cv2.VideoCapture(F"archive/videos/{videos_list[i]}.mp4")
        success = success, frame = cap.read()
        frame_number = 1
        while cap.isOpened() and success:
            frame = cv2.resize(frame, (600, 600))
            image, results = detect_keypoints(frame, holistic_model)
            draw_landmarks(image, results)

#                 cv2.imshow("cam", image)
    
            keypoints = extract_keypoints(results)
            save_path = os.path.join("archive/keypoints", word, str(i), f"frame_{frame_number}")
            np.save(save_path, keypoints)

            keypoints = None
            del keypoints

            success, frame = cap.read()
            frame_number += 1

#                 cv2.waitKey(1)

        cap.release()
        cv2.destroyAllWindows()
    end = datetime.datetime.now()
    print(f"\tprocess took about: {end-start}")

In [ ]:
holistic = mp_holistic_model.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [ ]:
create_np_keypoints_file(wlas_df.iloc[0]['gloss'], wlas_df.iloc[0]['videos_ids'], 0, holistic)

extracting the keypoints for (book) of index -> (0).
	extracting the keypoints from video number (0) ...


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video number (3) ...
	extracting the keypoints from video number (4) ...
	extracting the keypoints from video number (5) ...
	process took about: 0:00:20.948240


In [ ]:
start = datetime.datetime.now()
wlas_df.apply(lambda row: create_np_keypoints_file(row['gloss'], row['videos_ids'], row.name, holistic), axis=1)
end = datetime.datetime.now()
print(f"\tprocess took about: {end-start}")

extracting the keypoints for (book) of index -> (0).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video number (3) ...
	extracting the keypoints from video number (4) ...
	extracting the keypoints from video number (5) ...
	process took about: 0:00:20.908290
extracting the keypoints for (drink) of index -> (1).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video number (3) ...
	extracting the keypoints from video number (4) ...
	extracting the keypoints from video number (5) ...
	extracting the keypoints from video number (6) ...
	extracting the keypoints from video number (7) ...
	extracting the keypoints from video number (8) ...
	extracting the keypoints from video number (9) ...
	extracting the keypoints

	extracting the keypoints from video number (7) ...
	extracting the keypoints from video number (8) ...
	process took about: 0:00:33.364034
extracting the keypoints for (help) of index -> (12).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video number (3) ...
	extracting the keypoints from video number (4) ...
	extracting the keypoints from video number (5) ...
	extracting the keypoints from video number (6) ...
	extracting the keypoints from video number (7) ...
	extracting the keypoints from video number (8) ...
	extracting the keypoints from video number (9) ...
	extracting the keypoints from video number (10) ...
	extracting the keypoints from video number (11) ...
	extracting the keypoints from video number (12) ...
	extracting the keypoints from video number (13) ...
	process took about: 0:00:47.395689
extracting the keypoints for (no) of i

	extracting the keypoints from video number (8) ...
	extracting the keypoints from video number (9) ...
	process took about: 0:00:36.688244
extracting the keypoints for (many) of index -> (24).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video number (3) ...
	extracting the keypoints from video number (4) ...
	extracting the keypoints from video number (5) ...
	extracting the keypoints from video number (6) ...
	extracting the keypoints from video number (7) ...
	extracting the keypoints from video number (8) ...
	extracting the keypoints from video number (9) ...
	process took about: 0:00:32.635190
extracting the keypoints for (mother) of index -> (25).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video n

	extracting the keypoints from video number (9) ...
	extracting the keypoints from video number (10) ...
	process took about: 0:00:41.734293
extracting the keypoints for (family) of index -> (37).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video number (3) ...
	extracting the keypoints from video number (4) ...
	extracting the keypoints from video number (5) ...
	extracting the keypoints from video number (6) ...
	extracting the keypoints from video number (7) ...
	extracting the keypoints from video number (8) ...
	extracting the keypoints from video number (9) ...
	extracting the keypoints from video number (10) ...
	process took about: 0:00:44.177179
extracting the keypoints for (fish) of index -> (38).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video

	extracting the keypoints from video number (7) ...
	extracting the keypoints from video number (8) ...
	extracting the keypoints from video number (9) ...
	process took about: 0:00:35.421387
extracting the keypoints for (wrong) of index -> (50).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video number (3) ...
	extracting the keypoints from video number (4) ...
	extracting the keypoints from video number (5) ...
	extracting the keypoints from video number (6) ...
	extracting the keypoints from video number (7) ...
	process took about: 0:00:26.133905
extracting the keypoints for (accident) of index -> (51).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video number (3) ...
	extracting the keypoints from vide

...

	extracting the keypoints from video number (3) ...
	extracting the keypoints from video number (4) ...
	extracting the keypoints from video number (5) ...
	process took about: 0:00:28.238271
extracting the keypoints for (precise) of index -> (1874).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video number (3) ...
	process took about: 0:00:15.646229
extracting the keypoints for (preschool) of index -> (1875).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	process took about: 0:00:11.373751
extracting the keypoints for (pride) of index -> (1876).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video nu

	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video number (3) ...
	process took about: 0:00:20.986727
extracting the keypoints for (recommend) of index -> (1901).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video number (3) ...
	process took about: 0:00:14.603633
extracting the keypoints for (recover) of index -> (1902).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video number (3) ...
	extracting the keypoints from video number (4) ...
	extracting the keypoints from video number (5) ...
	process took about: 0:00:25.380731
extracting the keypoints for (regular) of index -> (1903).
	extracting the keypoints from video 

	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video number (3) ...
	process took about: 0:00:15.135879
extracting the keypoints for (shout) of index -> (1927).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	process took about: 0:00:11.196841
extracting the keypoints for (shovel) of index -> (1928).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	process took about: 0:00:10.935263
extracting the keypoints for (sin) of index -> (1929).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video number (3) ...
	process took about: 0:00:15.641220
extracting

	extracting the keypoints from video number (3) ...
	process took about: 0:00:12.329555
extracting the keypoints for (sunshine) of index -> (1954).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video number (3) ...
	extracting the keypoints from video number (4) ...
	process took about: 0:00:19.354167
extracting the keypoints for (superman) of index -> (1955).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video number (3) ...
	process took about: 0:00:16.430002
extracting the keypoints for (surrender) of index -> (1956).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from vide

	process took about: 0:00:07.431664
extracting the keypoints for (u) of index -> (1980).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video number (3) ...
	process took about: 0:00:14.998968
extracting the keypoints for (unique) of index -> (1981).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	extracting the keypoints from video number (3) ...
	extracting the keypoints from video number (4) ...
	extracting the keypoints from video number (5) ...
	process took about: 0:00:23.063716
extracting the keypoints for (upstairs) of index -> (1982).
	extracting the keypoints from video number (0) ...
	extracting the keypoints from video number (1) ...
	extracting the keypoints from video number (2) ...
	process took about: 0:00:08.721234
extractin